In [55]:
%pylab inline
from mnist import MNIST
from sklearn import metrics
from scipy import linalg
from numba import jit
from sklearn.kernel_approximation import RBFSampler, Nystroem


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
mndata = MNIST('/data/vaishaal/mnist')

In [3]:
train_data = mndata.load_training()
test_data = mndata.load_testing()


In [24]:
N = 60000

In [25]:
X_train = np.array(train_data[0])[:N, :].astype('float32')/255.0
y_train = np.array(train_data[1])[:N].astype('float32')
X_test = np.array(test_data[0])[:N, :].astype('float32')/255.0
y_test = np.array(test_data[1])[:N].astype('float32')
y_train_one_hot = np.eye(10)[:N,:][y_train.astype('int')]
y_test_one_hot = np.eye(10)[:N,:][y_test.astype('int')]


In [26]:
@jit
def fast_exp(K):
    for i in range(K.shape[0]):
        for j in range(K.shape[1]):
            K[i,j] = math.exp(K[i,j])
    return K

def computeDistanceMatrix(XTest, XTrain):
    XTrain = XTrain.reshape(XTrain.shape[0], -1)
    XTest = XTest.reshape(XTest.shape[0], -1)
    XTrain_norms = (np.linalg.norm(XTrain, axis=1) ** 2)[:, np.newaxis]
    XTest_norms = (np.linalg.norm(XTest, axis=1) ** 2)[:, np.newaxis]
    K = XTest.dot(XTrain.T)
    K *= -2
    K += XTrain_norms.T
    K += XTest_norms
    return K

def computeRBFGramMatrix(XTest, XTrain, gamma=1):
    gamma = -1.0 * gamma
    return fast_exp(gamma*computeDistanceMatrix(XTest, XTrain))


In [27]:
lambdav = 1e-5
%time K = computeRBFGramMatrix(X_train, X_train, gamma=1e-3)

CPU times: user 3min 2s, sys: 37.5 s, total: 3min 39s
Wall time: 1min 17s


In [28]:
%time K_test = computeRBFGramMatrix(X_test, X_train, gamma=1e-3)
diag = np.diag_indices(K.shape[0])
K[diag] += lambdav
%time model = linalg.solve(K, y_train_one_hot, sym_pos=True)
K[diag] -= lambdav
%time y_train_pred = np.argmax(K.dot(model), axis=1)
%time train_acc = metrics.accuracy_score(y_train, y_train_pred)
%time y_test_pred = np.argmax(K_test.dot(model), axis=1)
%time test_acc = metrics.accuracy_score(y_test, y_test_pred)
print("Train accuracy is ", train_acc)
print("Test accuracy is ", test_acc)

CPU times: user 1min 7s, sys: 10.5 s, total: 1min 17s
Wall time: 11.5 s
CPU times: user 1h 45min 13s, sys: 17.5 s, total: 1h 45min 31s
Wall time: 1min 55s
CPU times: user 1min 10s, sys: 7.61 s, total: 1min 18s
Wall time: 12.8 s
CPU times: user 368 ms, sys: 4 ms, total: 372 ms
Wall time: 10.4 ms
CPU times: user 15.5 s, sys: 1.79 s, total: 17.3 s
Wall time: 1.98 s
CPU times: user 1.18 s, sys: 4 ms, total: 1.18 s
Wall time: 20.4 ms
('Train accuracy is ', 1.0)
('Test accuracy is ', 0.9839)


In [41]:
rbf_sampler = RBFSampler(gamma=1e-3, n_components=int(8192))
rbf_sampler.fit(X_train)
X_train_lift = rbf_sampler.transform(X_train)


In [56]:
nystroem = Nystroem(gamma=1e-3, n_components=int(8192))
nystroem.fit(X_train)
X_train_lift = nystroem.transform(X_train)

In [57]:
ztz = X_train_lift.T.dot(X_train_lift)

In [59]:
lambdav = 1e-3

In [60]:
ztz_reg = ztz + lambdav * np.eye(ztz.shape[0])

In [61]:
L = np.linalg.cholesky(ztz_reg)

In [62]:
U = linalg.solve(L, X_train_lift.T)

In [63]:
def make_smart_precondition(U, lambdav):
    def prc(x):
        return (1.0/lambdav)*(x - U.T.dot(U.dot(x)))
    return prc

In [64]:
prc = make_smart_precondition(U, lambdav)

In [49]:
def pcg(A,b,prc=lambda x: x, max_iter=100, tol=1e-3):
    i = 0 
    x = np.zeros((A.shape[0], b.shape[1]))
    r = b - A.dot(x)
    d = prc(r)
    delta_new = np.linalg.norm(r.T.dot(d))
    delta_0 = delta_new
    while (True):
        if (i > max_iter):
            break
            
        if (delta_new < tol*delta_0):
            break
        # Expensive
        q = A.dot(d)
        a = delta_new/np.linalg.norm(d.T.dot(q))
        print(a)
        x = x + a*d
        if (i % 20 == 0):
            r = b - A.dot(x)
        else:
            r = r - a*q
        print("Iter {0}, NORM IS {1}".format(i,np.linalg.norm(r)))
        s = prc(r)
        delta_old = delta_new
        delta_new = np.linalg.norm(r.T.dot(s))
        beta = delta_new/delta_old
        d = s + beta * d
        i = i + 1
        
    return x

In [66]:
lambdav = 1e-4
diag = np.diag_indices(K.shape[0])
K[diag] += lambdav
%time model_pcg = pcg(K, y_train_one_hot, prc=prc, tol=1e-8)
K[diag] -= lambdav
%time y_train_pred_pcg = np.argmax(K.dot(model_pcg), axis=1)
%time train_acc_pcg = metrics.accuracy_score(y_train, y_train_pred_pcg)
%time y_test_pred_pcg = np.argmax(K_test.dot(model_pcg), axis=1)
%time test_acc_pcg = metrics.accuracy_score(y_test, y_test_pred_pcg)
print("train", train_acc_pcg)
print("test", test_acc_pcg)

0.427935465476
Iter 0, NORM IS 172.585521382
0.233987469583
Iter 1, NORM IS 147.286227611
0.220515427451
Iter 2, NORM IS 121.680271803
0.18911861914
Iter 3, NORM IS 90.9649462989
0.191798083928
Iter 4, NORM IS 64.1338569689
0.223469005632
Iter 5, NORM IS 43.8591329833
0.224294187013
Iter 6, NORM IS 29.7707836764
0.225764341967
Iter 7, NORM IS 19.8907038127
0.23510759686
Iter 8, NORM IS 12.6367083305
0.237052369957
Iter 9, NORM IS 7.66189253531
0.239135278928
Iter 10, NORM IS 4.53282214894
0.241992418019
Iter 11, NORM IS 3.16491866023
0.28640988504
Iter 12, NORM IS 3.16904420299
0.325464618897
Iter 13, NORM IS 3.53064535386
0.319041943235
Iter 14, NORM IS 3.73748838099
0.317217908103
Iter 15, NORM IS 3.41998280042
0.330187506501
Iter 16, NORM IS 2.60835553223
0.323197209974
Iter 17, NORM IS 2.07107251871
0.296512927976
Iter 18, NORM IS 2.03494879847
0.267447319972
Iter 19, NORM IS 1.98714636248
0.268147308583
Iter 20, NORM IS 1.64246125549
0.278730534419
Iter 21, NORM IS 1.21707887032
0

array([[ 12.11929162,   1.00612057, -23.08249818, ..., -16.16399858,
         32.46745129,   6.1376664 ],
       [ 21.28909037,  15.13471109,  11.59349335, ..., -21.67821197,
        -42.15139715, -34.35247609],
       [ 16.08034455,   1.51985261, -10.84489735, ...,  -2.17286364,
         -4.63042263,  37.44360878],
       ..., 
       [-26.7195863 , -21.95883457,  10.77941463, ...,  21.97723103,
        -22.39407978, -15.22387314],
       [-43.06032465,  -2.12399655, -17.81350485, ..., -24.19085428,
        -32.41879885,  16.42227703],
       [-13.72993714, -13.2217883 , -22.21293338, ...,   0.96924659,
         43.39213973,  17.48130907]])

In [130]:
model_pcg

array([[ 16.6915398 ,   7.75003048, -32.37554295, ..., -13.07163348,
         45.97279594,  12.26993159],
       [ 26.37911725,  31.90717808,  18.36265749, ..., -37.67729199,
        -53.12838866, -72.47751839],
       [ 18.00396607,  -3.24778434,  -9.45986685, ...,   3.2445086 ,
         -7.99444691,  49.46757378],
       ..., 
       [-81.27319881, -39.88923286,  47.41878124, ...,  20.73184882,
        -34.52863183, -33.10110771],
       [-56.82578615, -11.89142043, -28.38461232, ..., -39.20771207,
        -42.33527191,  30.28163667],
       [-26.07937586,  -5.3022592 , -26.90940086, ...,  10.23020306,
         82.35308535,  24.48517809]])

In [187]:
np.linalg.norm(K.dot(model) - y_train_one_hot)

7.1071946180237653

In [193]:
diag = np.diag_indices(K.shape[0])
K[diag] += lambdav
model_pcg = pcg(K, y_train_one_hot, prc)
K[diag] -= lambdav



Iter 0, NORM IS 8373.90311975
Iter 2, NORM IS 3512.70395896
Iter 4, NORM IS 4098.52870757
Iter 6, NORM IS 3040.06515998
Iter 8, NORM IS 1911.03286012
Iter 10, NORM IS 2920.61537079
Iter 12, NORM IS 2284.05415358


KeyboardInterrupt: 

In [283]:
C = np.random.randn(100, 100)
A = C.dot(C.T)
b = np.random.randn(100)[:, np.newaxis]
x_star = linalg.solve(A,b, sym_pos=True)
best_loss = np.linalg.norm(A.dot(x_star) - b)
print("Best Loss {0}".format(best_loss))



Best Loss 1.35445623455e-12


In [287]:
x_hat= pcg(A.astype('float64'), b.astype('float64'), max_iter=500, tol=1e-8, prc=make_jacobi(A))


[[ 0.99098699]]
Iter 0, NORM IS 11.2308027437
[[ 0.90530137]]
Iter 1, NORM IS 11.7695625114
[[ 0.8677003]]
Iter 2, NORM IS 10.1201055657
[[ 1.01794591]]
Iter 3, NORM IS 9.99866759022
[[ 0.97513655]]
Iter 4, NORM IS 9.8048727723
[[ 1.04644764]]
Iter 5, NORM IS 9.11470792598
[[ 0.90148744]]
Iter 6, NORM IS 9.02572371894
[[ 0.9786414]]
Iter 7, NORM IS 8.19567561895
[[ 1.27359459]]
Iter 8, NORM IS 9.23396343101
[[ 1.0882563]]
Iter 9, NORM IS 7.81994169029
[[ 1.11671018]]
Iter 10, NORM IS 7.03060781678
[[ 1.50916709]]
Iter 11, NORM IS 9.15600478555
[[ 1.0572362]]
Iter 12, NORM IS 8.46698528519
[[ 1.30749204]]
Iter 13, NORM IS 9.37837960348
[[ 0.9425041]]
Iter 14, NORM IS 8.92225367645
[[ 1.090044]]
Iter 15, NORM IS 9.0294245054
[[ 0.90858848]]
Iter 16, NORM IS 7.15908258985
[[ 1.07009403]]
Iter 17, NORM IS 6.85877337402
[[ 1.2126261]]
Iter 18, NORM IS 6.47411917675
[[ 1.09948019]]
Iter 19, NORM IS 5.43949561152
[[ 1.16770143]]
Iter 20, NORM IS 5.17414988882
[[ 1.27957141]]
Iter 21, NORM IS 

In [272]:
np.linalg.eigh(A)[0][-1]/np.linalg.eigh(A)[0][0]


120004.79880076343

In [330]:
prc = make_smart_precondition(U, lambdav)


In [332]:
K[diag] += lambdav
K_precond = prc(K)
K[diag] -= lambdav


In [278]:
make_jacobi = lambda A: lambda x: np.diag(1.0/np.diag(A)).dot(x)

In [335]:
K

array([[ 1.00002003,  0.91609752,  0.88845819, ...,  0.93980008,
         0.909823  ,  0.9062078 ],
       [ 0.91609752,  1.00002003,  0.87876332, ...,  0.8972308 ,
         0.93461108,  0.91334242],
       [ 0.88845819,  0.87876332,  1.00002003, ...,  0.90333456,
         0.90644521,  0.91350317],
       ..., 
       [ 0.93980008,  0.8972308 ,  0.90333456, ...,  1.00002003,
         0.91644579,  0.92249513],
       [ 0.909823  ,  0.93461108,  0.90644521, ...,  0.91644579,
         1.00002003,  0.92091209],
       [ 0.9062078 ,  0.91334242,  0.91350317, ...,  0.92249513,
         0.92091209,  1.00002003]], dtype=float32)

In [336]:
K_precond

array([[ 126.32953079,   24.80131255,   18.53930662, ...,   10.45374752,
          17.15854103,   17.58860035],
       [ -27.38576808,   14.67955008,  -13.2925559 , ...,  -17.38688579,
         -31.37449067,  -13.64780354],
       [ -24.94962828,  -13.68118589,  135.13791946, ...,  -27.84422298,
         -11.66058836,  -16.80828129],
       ..., 
       [  12.06810968,   19.23483916,    6.54706488, ...,   89.63225298,
          13.21332112,   -0.53846274],
       [ -55.77140386,  -62.8705892 ,  -50.24562781, ...,  -67.55667326,
          42.8061752 ,  -62.03363452],
       [  31.17048165,   44.00313194,   42.51842822, ...,   37.24262983,
          38.88636276,  140.41297419]])

(array([  0.43124252,  -0.59161803,  -4.43608497,   0.40382292,
          2.59269597,   2.08581895,  -3.28563846,   0.05231108,
         -5.8735954 ,  -1.52592873,  -6.51319883,  -0.22595505,
         -5.36357019,  -0.15650906,   5.32313735,  -1.00757367,
         -5.28666333,  -1.86669584,  -0.8228179 ,   2.53977234,
         -4.61418422,   2.60228249, -10.9269566 ,  -5.23378378,
         -1.76105112,   6.26615143,  -7.06527465,   0.84729513,
         -4.80899157,  -0.03069856,  -9.53908601,  -7.82306194,
          4.61542938,  -1.80526501,   1.2188593 ,  -3.36736884,
          2.31664223,   1.49768499,   3.31248111, -10.00050497,
          0.76805315,  -5.56378985,  -0.33040718,  -3.0503181 ,
          3.58824832,  -1.85064305,   3.53101908,  -1.32833745,
         -5.62645798,  -4.37651291,  -1.85382498,   6.00855843,
         -1.28122811,   7.03567425,  -1.98708793,   3.97086537,
          9.92192469,   3.14431783,  -4.07153846,  10.11675898,
          9.89657684,   0.70097871,   2.

In [320]:
x_star

array([[  0.43124247],
       [ -0.59161792],
       [ -4.43608521],
       [  0.40382382],
       [  2.59269636],
       [  2.08581862],
       [ -3.28563838],
       [  0.05231155],
       [ -5.87359503],
       [ -1.52592856],
       [ -6.51319878],
       [ -0.22595468],
       [ -5.3635701 ],
       [ -0.15650947],
       [  5.32313707],
       [ -1.00757386],
       [ -5.28666412],
       [ -1.86669614],
       [ -0.82281763],
       [  2.53977249],
       [ -4.61418437],
       [  2.60228219],
       [-10.9269571 ],
       [ -5.23378387],
       [ -1.76105139],
       [  6.26615149],
       [ -7.06527434],
       [  0.84729505],
       [ -4.80899214],
       [ -0.03069899],
       [ -9.53908438],
       [ -7.8230622 ],
       [  4.61542936],
       [ -1.80526483],
       [  1.21885889],
       [ -3.36736863],
       [  2.31664184],
       [  1.49768468],
       [  3.31248095],
       [-10.00050497],
       [  0.76805347],
       [ -5.56378956],
       [ -0.33040758],
       [ -3

In [324]:
K_approx = X_train_lift.dot(X_train_lift.T)

In [326]:
K_approx

array([[ 1.00036362,  0.91586422,  0.893553  , ...,  0.94035426,
         0.91390355,  0.91031235],
       [ 0.91586422,  0.99813358,  0.88444416, ...,  0.89827786,
         0.93629677,  0.91488062],
       [ 0.893553  ,  0.88444416,  1.00594779, ...,  0.90540712,
         0.91315245,  0.92042929],
       ..., 
       [ 0.94035426,  0.89827786,  0.90540712, ...,  1.00202177,
         0.91896751,  0.9253971 ],
       [ 0.91390355,  0.93629677,  0.91315245, ...,  0.91896751,
         1.00401308,  0.92503986],
       [ 0.91031235,  0.91488062,  0.92042929, ...,  0.9253971 ,
         0.92503986,  1.00409796]])

In [327]:
K

array([[ 1.00002003,  0.91609752,  0.88845819, ...,  0.93980008,
         0.909823  ,  0.9062078 ],
       [ 0.91609752,  1.00002003,  0.87876332, ...,  0.8972308 ,
         0.93461108,  0.91334242],
       [ 0.88845819,  0.87876332,  1.00002003, ...,  0.90333456,
         0.90644521,  0.91350317],
       ..., 
       [ 0.93980008,  0.8972308 ,  0.90333456, ...,  1.00002003,
         0.91644579,  0.92249513],
       [ 0.909823  ,  0.93461108,  0.90644521, ...,  0.91644579,
         1.00002003,  0.92091209],
       [ 0.9062078 ,  0.91334242,  0.91350317, ...,  0.92249513,
         0.92091209,  1.00002003]], dtype=float32)

In [58]:
ztz.shape

(8192, 8192)